# CNN_Train

## Imports

In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../modules'))
import notebook_loading

from CNN_Data import Data_Creator

import io
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

importing Jupyter notebook from CNN_Data.ipynb
